In [1]:
import numpy as np
import uproot as ur
import awkward as ak
import time as t
import os
import argparse
from copy import deepcopy
path_prefix = 'D:/Work/EPE/ML4pi/'
plotpath = path_prefix+'plots/'
modelpath_c = path_prefix+''
modelpath = path_prefix+''
data_path = path_prefix + "v7/"
ext_path = "H:/EPE_file_storage/"
ext_modelpath = ext_path + "Model/"
ext_datapath = 'H:/EPE_file_storage/data_storage/pipm/root/'
stmc_v2_path = ext_datapath + 'STMC_v2/'

In [2]:
import h5py as h5

In [3]:
Y = np.load(stmc_v2_path + 'Y_large.npy')
Y.shape

(2500000, 3)

In [4]:
X = np.load(stmc_v2_path+'STMC_v2_X_'+str(40)+'_files.npy')
X

(411495, 1389, 6)

In [6]:
def normalized_data(X,Y):
    ''' 
    Normalizes the data
    '''
    t0 = t.time()
    # take log of Y
    target_zero_mask = Y[:,0] > .005
    Ylog = np.log(Y)[target_zero_mask]
    
    ## Normalize rPerp to 1/3630
    rPerp_mask = X[:,:,3] != 0
    X[:,:,3][rPerp_mask] = X[:,:,3][rPerp_mask]/3630.

    isnan = np.isnan(X[:,:,0])
    anytruth = np.any(isnan)
    print(anytruth)

    ## Energy Values that are not zero!
    E_nonZeroMask = X[:,:,0] != 0
    X[:,:,0][E_nonZeroMask] = np.log(X[:,:,0][E_nonZeroMask])
    cellE_mean = np.mean(X[:,:,0][E_nonZeroMask])
    cellE_std = np.std(X[:,:,0][E_nonZeroMask])
    X[:,:,0][E_nonZeroMask] = (X[:,:,0][E_nonZeroMask] - cellE_mean)/cellE_std

    ## Eta and Phi
    # do nothing for now as a control and check performance
    eta_mask = X[:,:,1] != 0
    X[:,:,1][eta_mask] = X[:,:,1][eta_mask]/2.5

    phi_mask = X[:,:,2] != 0
    cellPhi_std = np.std(X[:,:,2][phi_mask])
    X[:,:,2][phi_mask] = X[:,:,2][phi_mask]/cellPhi_std
    t1 = t.time()
    print('Time to Normalize: '+str(t1-t0)+' (s)')
    return X, Ylog

In [7]:
X_norm, Y_norm = normalized_data(X, Y[:411495,:])

C:\Users\alber\AppData\Local\Temp/ipykernel_196960/1096770626.py:8: RuntimeWarning: divide by zero encountered in log
  Ylog = np.log(Y)[target_zero_mask]


False
Time to Normalize: 20.999014616012573 (s)


In [9]:
X_norm.shape

(411495, 1389, 6)

In [10]:
Y_norm.shape

(411495, 3)

In [11]:
# train val split
import sklearn.model_selection as skms
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_norm, Y_norm, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=42)

with h5.File(stmc_v2_path + 'STMC_v2_train.h5', 'w') as f:
    f.create_dataset('X', data=X_train)
    f.create_dataset('Y', data=Y_train)
with h5.File(stmc_v2_path + 'STMC_v2_val.h5', 'w') as f:
    f.create_dataset('X', data=X_val)
    f.create_dataset('Y', data=Y_val)
with h5.File(stmc_v2_path + 'STMC_v2_test.h5', 'w') as f:
    f.create_dataset('X', data=X_test)
    f.create_dataset('Y', data=Y_test)